# Gün 2 bölüm 1: Bir SIR'in yapı taşları

<div style="background: #f8f9fa; padding: 0.5rem 1rem; border-radius: 8px; margin-bottom: 1rem; border-left: 4px solid #007bff;">
<a href="../../index.ipynb" style="text-decoration: none; color: #007bff; font-weight: bold;">← Kurs İndeksine Geri Dön</a> | 
<a href="../../index-tr.ipynb" style="text-decoration: none; color: #dc3545; font-weight: bold;">🇹🇷 Türkçe İndeks</a>
</div>

---

Artık bir bölmeli modeli nasıl kodlayacağımızı biliyoruz. Ayrıca SIR modelinin ne olduğunu ve ana yapı taşlarının neler olduğunu biliyoruz. Bu egzersizde tamamen dinamik bir SIR modeli oluşturmak için önceki kodumuz üzerine inşa edeceğiz.   

## 1. Bir SIR modeli oluşturmak 

### Vaka çalışması: şehrinizde yeni bir X2021 virüsü tanımlandı ve şu anda bir salgına neden oluyor. Bu salgının potansiyel boyutu hakkında geniş bir fikir edinmek için modelleme projeksiyonları sağlamak üzere DSÖ tarafından görevlendirildiniz. Devam eden epidemiyolojik araştırma, model geliştirmenizde size rehberlik edebilecek aşağıdaki gerçekleri tespit etmiştir:    

1)	Virüs solunum yolundan mikro damlacıklar yoluyla insandan insana bulaşır.    

2)	Enfeksiyon yerleştikten sonra enfekte bireyler ortalama 6 gün boyunca bulaştırıcı kalırlar    

3)	Komşu bir şehirdeki önceki X2021 salgınından CFR'nin ~%15 olduğunu biliyoruz    

4)	Herhangi bir belirli popülasyon grubunda artmış duyarlılık için bilinen faktörler tanımlanmamıştır   

5)	Aynı önceki salgından, günde 0,5 enfeksiyon oranı (CI95% 0,3 - 0,6) tahmin edilmiştir
Görev: Aşağıdaki kodu kullanarak (önceki oturumumuzdan) yukarıdaki vakayı yansıtan bir SIR modeli oluşturun ve aşağıdaki soruları cevaplamaya çalışın:   

    1)	X2021 salgınının ne zaman zirve yapmasını bekliyoruz (gün cinsinden)?
    2)	Salgının sonunda kaç kişinin enfekte olmasını bekliyoruz?
    3)	Enfeksiyon oranındaki belirsizliğe dayalı olarak salgının nihai boyutu (enfekte sayısı) ve beklenen zirve (gün cinsinden) için bir aralık verebilir misiniz?


Not: Aşağıdaki kodu R Studio oturumunuza kopyalayın. ?? ile işaretlenmiş boşlukları doldurmaya çalışın   

In [ ]:
# PAKETLERİ YÜKLE:
library(deSolve)
library(reshape2)
library(ggplot2)
library(here)

rm(list = ls())
setwd(here())
# Popülasyon büyüklüğü:
N <- 1000

# MODEL GİRİŞLERİ:

# Başlangıç koşulları
initial_state_values <- c(
  S = N-1,
  I = 1,
  R = 0,
  M = 0)           

# Günlük parametre değerleri
  CFR<- ?? # <---------------------- Boşluğu doldur
   
  gamma <- ?? # İyileşme oranı # <---------------------- Boşluğu doldur
  
  beta <- ??  # enfeksiyon oranı # <---------------------- Boşluğu doldur
  
  mu <- ?? ## # <---------------------- Boşluğu doldur
  
  parameters <- c(beta, gamma, mu)


# MODEL FONKSİYONU: 
times <- seq(from = 0, to = 150, by = 1)   # 0'dan 365 güne kadar günlük aralıklarla

sir_model <- function(time, state, parameters) {  
  
  with(as.list(c(state, parameters)), {     
    
    # Toplam popülasyon büyüklüğü N'yi hesaplama (her bölmedeki kişi sayısının toplamı)
    
    N <- S+I+R
    
    # FOI'yi hesaplama
    # Enfeksiyon kuvvetini nasıl tanımlayabiliriz? #
    
    lambda <- ??  #   <---------------------- Boşluğu doldur
      
      # Diferansiyel denklemler
      dS <-  - lambda * S # S için diferansiyel denklemi yazabilir misin? 
      dI <- lambda * S - I*(gamma+mu)     
    dR <- gamma * I
    dM <- I*mu
    
    return(list(c(dS, dI, dR, dM))) 
  })
  
}



# MODEL ÇIKTISI (diferansiyel denklemleri çözme):

# ode entegrasyon algoritmasını kullanarak diferansiyel denklemleri çözme
output <- as.data.frame(ode(y = initial_state_values, 
                            times = times, 
                            func = sir_model,
                            parms = parameters))



# çıktı veri setini uzun formata çevir
output_long <- melt(as.data.frame(output), id = "time")                 


# S, I ve R bölmelerindeki kişi sayısını zaman içinde çizdir
ggplot(data = output_long,                                               # çizilecek veriyi içeren nesneyi belirt
       aes(x = time, y = value, colour = variable, group = variable)) +  # sütunları eksenlere ve gruplara ata
  geom_line(size=2) +                                                          # veriyi çizgiler olarak göster
  xlab("Zaman (gün)")+                                                   # x ekseni için etiket ekle
  ylab("Sayı") +                                                       # y ekseni için etiket ekle
  labs(colour = "Bölme",                                           # lejand başlığı ekle  
       title = "SIR modeli")    



# Bireylerin oranını çizdir 

output2_long<- output_long
output2_long$value<-output_long$value/N


# S, I ve R bölmelerindeki insanların oranını zaman içinde çizdir
ggplot(data = output2_long,                                               # çizilecek veriyi içeren nesneyi belirt
       aes(x = time, y = value, colour = variable, group = variable)) +  # sütunları eksenlere ve gruplara ata
  geom_line(size=2) +                                                          # veriyi çizgiler olarak göster
  xlab("Zaman (gün)")+                                                   # x ekseni için etiket ekle
  ylab("Sayı") +                                                       # y ekseni için etiket ekle
  labs(colour = "Bölme",                                           # lejand başlığı ekle  
       title = "SIR modeli")   


## SIR varsayımlarının analizi

Dersimizde gözden geçirilen ana varsayımları ve kavramları düşünün ve şunları cevaplamaya çalışın:   

1)	Enfeksiyon riskinin homojen varsayımı neden bir basitleştirmedir ve hangi faktörler bunu etkileyebilir? Potansiyel heterojenlik kaynakları nelerdir?   

2)	Hangi faktörler popülasyon genelinde homojen p varsayımımızı etkileyebilir?    


